In [ ]:
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine

In [ ]:
# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import numpy as np
import folium 
import geopandas as gpd
import math
from tqdm.notebook import tqdm
import utm
from shapely.geometry import Point, Polygon
from haversine import haversine
from plotnine import *

In [ ]:
# 돌봄센터 api인데 엉망
gpd.read_file('/content/drive/My Drive/Project/Seoul/돌봄시설 api/11101716_20201002181837.shp_point.shp', encoding = 'cp949')

,CONTS_NAME,IMG_URL,ADDR_OLD,ADDR_NEW,COORD_DATA,COORD_X,COORD_Y,TEL_NO,CONTS_DETL,CONTS_STAT,EX_DATA_01,EX_DATA_02,NAME_01,VALUE_01,NAME_02,VALUE_02,NAME_03,VALUE_03,NAME_04,VALUE_04,NAME_05,VALUE_05,NAME_06,VALUE_06,NAME_07,VALUE_07,NAME_08,VALUE_08,NAME_09,VALUE_09,NAME_10,VALUE_10,NAME_11,VALUE_11,NAME_12,VALUE_12,NAME_13,VALUE_13,NAME_14,VALUE_14,NAME_15,VALUE_15,NAME_16,VALUE_16,NAME_17,VALUE_17,NAME_18,VALUE_18,NAME_19,VALUE_19,NAME_20,VALUE_20,COORD_TYPE,LN_PATTERN,LN_COLOR,LN_WEIGHT,MOVIE_URL,VOICE_URL,BASE_AREA,POINT_NUM,LAN_TYPE,KEYWORD,SUB_ID,geometry
0,2020년 갤러리 구루지 기획전지 전승일의 오토마타(예술과 과학의 만남),None,None,가마산로25길 21,None,126.889860000,37.497210000,010-6839-1407,None,1,None,None,소개,일 시: 2020.1.29(수)~2.22(토)\n시 간: 10:00~1...,<div></div>,<div class='buttons'><a href='' target='_paren...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,#0000FF,2,None,None,04526,다사46064439,KOR,None,None,POINT (126.88986 37.49721)
1,강동어린이회관 휴관안내 (2/29일까지),None,None,성내로6길 16,None,127.122480000,37.529170000,None,None,1,None,None,소개,강동어린이회관 휴관안내\n\n강동어린이회관은 신종코로나 바이러스 감염증 예방 및 확...,<div></div>,<div class='buttons'><a href='' target='_paren...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,#0000FF,2,None,None,04526,다사66644782,KOR,None,None,POINT (127.12248 37.52917)
2,제38회 신수동 음악회 크리스마스선물,None,None,독막로 192,None,126.935210000,37.547070000,None,None,1,None,None,소개,제38회 신수동음악회\n\n클래식 테라피 앙상블 '뮤직하일렌'과 함께하는 사랑의 언...,<div></div>,<div class='buttons'><a href='' target='_paren...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,#0000FF,2,None,None,None,다사50104989,KOR,None,None,POINT (126.93521 37.54707)
3,사람사랑 치과,None,None,목동로 203,None,126.863820000,37.526710000,02-2605-2275,None,1,None,None,소개,"◎진료시간 : 평일 (수,금)10:00~19:00, 토요일 09:00~14:00(점...",<div></div>,<div class='buttons'><a href='//icare.seoul.go...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,#0000FF,2,None,None,04526,다사43784767,KOR,None,None,POINT (126.86382 37.52671)
4,시립보라매청소년수련관,None,None,여의대방로20길 33,None,126.918920000,37.495430000,02-834-7231~4,None,1,None,None,소개,"*이용시간 \r\n월~일 6:00~22:30\r\n\r\n다양한 문화공간, 시설대관",<div></div>,<div class='buttons'><a href='//icare.seoul.go...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,#0000FF,2,None,None,04526,다사48634417,KOR,None,None,POINT (126.91892 37.49543)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14540,"(상암,도화) 시간제보육'아띠'가 직접 체험한 『랜선 공원 산책』 - 서울에너지드림센터",None,None,상암산로1길 71,None,126.883620000,37.580520000,002-308-0202,None,1,None,None,소개,마포구육아종합지원센터 시간제보육 ‘아띠’가 직접 체험한 『랜선 공원 산책』 시리즈는...,<div></div>,<div class='buttons'><a href='' target='_paren...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,#0000FF,2,None,None,04526,다사45575363,KOR,None,None,POINT (126.88362 37.58052)
14541,우리아이 키성장 솔루션,None,None,덕릉로 859,None,127.083230000,37.672930000,002-930-1944,None,1,None,None,소개,- 우리아이 키성장 솔루션 -\n\n* 일 시 : 2020. 9. 17(목) 10...,<div></div>,<div class='buttons'><a href='' target='_paren...,None,None,None,None,None,None,None,None,None,None,None

In [ ]:
# 돌봄시설 데이터_수작업..
dolbom = pd.read_excel('/content/drive/My Drive/Project/Seoul/돌봄시설 api/돌봄시설_위치.xlsx')
dolbom

,자치구,시설명,주소,법정동,Latitude,Longitude
0,중랑구,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,신내동,37.617108,127.106766
1,중구,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,신당동,37.565088,127.021852
2,중구,중구4호점 우리동네키움센터(모든아이 약수센터),서울특별시 중구 동호로5길 19,신당동,37.551780,127.012577
3,중구,중구3호점 우리동네키움센터(모든아이 손기정센터),서울특별시 중구 손기정로 101,만리동1가,37.555665,126.964550
4,중구,중구2호점 우리동네키움센터(모든아이 중림센터),서울특별시 중구 서소문로6길 16,중림동,37.598391,126.918197
...,...,...,...,...,...,...
86,강동구,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,암사동,37.550551,127.131430
87,강남구,강남4호점 우리동네키움센터(미미위 세곡 키움센터),서울특별시 강남구 밤고개로 231,자곡동,37.473781,127.106570
88,강남구,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,청담동,37.520398,127.054127
89,강남구,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,역삼동,37.493925,127.040900


In [ ]:
df_code = pd.read_pickle('/content/drive/My Drive/Project/Seoul/code/행정동_법정동_매칭코드.pickle')
df_code

,행정동코드,시군구명,읍면동명,법정동코드,동리명,geometry,위도,경도
0,1111053000,종로구,사직동,1111010600,통의동,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",37.574108,126.970144
1,1111053000,종로구,사직동,1111010700,적선동,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",37.574108,126.970144
2,1111053000,종로구,사직동,1111011200,체부동,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",37.574108,126.970144
3,1111053000,종로구,사직동,1111011300,필운동,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",37.574108,126.970144
4,1111053000,종로구,사직동,1111011400,내자동,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",37.574108,126.970144
...,...,...,...,...,...,...,...,...
737,1174057000,강동구,암사제1동,1174010700,암사동,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ...",37.551855,127.134586
738,1174061000,강동구,천호제2동,1174010900,천호동,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ...",37.544691,127.122275
739,1174068500,강동구,길동,1174010500,길동,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ...",37.539725,127.145907
740,1153078000,구로구,오류제2동,1153010800,오류동,"MULTIPOLYGON (((126.83179 37.47757, 126.83235 ...",37.485265,126.838372


In [ ]:
dong_boundary['행정동'] = dong_boundary['행정동'].astype(str)
code['읍면동명'] = code['읍면동명'].astype(str)

In [ ]:
# 법정동 행정동 매칭 코드로 병합 --> 91개가 너무 불어남... 
dolbom.merge(df_code, left_on ='법정동', right_on = '동리명')

,자치구,시설명,주소,법정동,Latitude,Longitude,행정동코드,시군구명,읍면동명,법정동코드,동리명,geometry,위도,경도
0,중랑구,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,신내동,37.617108,127.106766,1126068000,중랑구,신내1동,1126010600,신내동,"MULTIPOLYGON (((127.11567 37.61929, 127.11697 ...",37.612802,127.104507
1,중랑구,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,신내동,37.617108,127.106766,1126069000,중랑구,신내2동,1126010600,신내동,"MULTIPOLYGON (((127.09580 37.60061, 127.09574 ...",37.610951,127.091616
2,중구,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,신당동,37.565088,127.021852,1114065000,중구,신당제5동,1114016200,신당동,"MULTIPOLYGON (((127.02362 37.56517, 127.02671 ...",37.563581,127.022016
3,중구,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,신당동,37.565088,127.021852,1114061500,중구,신당동,1114016200,신당동,"MULTIPOLYGON (((127.01807 37.56557, 127.01599 ...",37.565748,127.013775
4,중구,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,신당동,37.565088,127.021852,1114062500,중구,다산동,1114016200,신당동,"MULTIPOLYGON (((127.01285 37.56058, 127.01364 ...",37.554703,127.008263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,강남구,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,역삼동,37.493925,127.040900,1168064000,강남구,역삼1동,1168010100,역삼동,"MULTIPOLYGON (((127.04718 37.50397, 127.04277 ...",37.500889,127.035491
309,강남구,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,역삼동,37.493925,127.040900,1168065000,강남구,역삼2동,1168010100,역삼동,"MULTIPOLYGON (((127.05251 37.49705, 127.05273 ...",37.498502,127.044891
310,강남구,강남1호점 우리동네키움센터(강남 다함께키움센터(태화)),서울특별시 강남구 광평로 185,수서동,37.484749,127.093420,1168072000,강남구,일원본동,1168011500,수서동,"MULTIPOLYGON (((127.09299 37.47726, 127.09194 ...",37.482370,127.084675
311,강남구,강남1호점 우리동네키움센터(강남 다함께키움센터(태화)),서울특별시 강남구 광평로 185,수서동,37.484749,127.093420,1168073000,강남구,일원1동,1168011500,수서동,"MULTIPOLYGON (((127.09674 37.49583, 127.10197 ...",37.493386,127.089246


http://www.s9soft.co.kr/single/s.html   

이 사이트 이용, 행정동 수작업 매칭

In [ ]:
# 행정동과 법정동 mapping code & 위치 정보 합치기 
df_dolbom = pd.read_excel('/content/drive/My Drive/Project/Seoul/돌봄시설 api/돌봄시설_위치_행정동.xlsx')
df_dolbom

,자치구,시설명,주소,법정동,행정동,Latitude,Longitude
0,중랑구,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,신내동,일원본동,37.617108,127.106766
1,중구,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,신당동,신당제5동,37.565088,127.021852
2,중구,중구4호점 우리동네키움센터(모든아이 약수센터),서울특별시 중구 동호로5길 19,신당동,약수동,37.551780,127.012577
3,중구,중구3호점 우리동네키움센터(모든아이 손기정센터),서울특별시 중구 손기정로 101,만리동1가,중림동,37.555665,126.964550
4,중구,중구2호점 우리동네키움센터(모든아이 중림센터),서울특별시 중구 서소문로6길 16,중림동,중림동,37.598391,126.918197
...,...,...,...,...,...,...,...
86,강동구,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,암사동,암사제1동,37.550551,127.131430
87,강남구,강남4호점 우리동네키움센터(미미위 세곡 키움센터),서울특별시 강남구 밤고개로 231,자곡동,세곡동,37.473781,127.106570
88,강남구,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,청담동,청담동,37.520398,127.054127
89,강남구,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,역삼동,역삼2동,37.493925,127.040900


In [ ]:
df_dolbom.columns

Index(['자치구', '시설명', '주소', '법정동', '행정동', 'Latitude', 'Longitude'], dtype='object')

In [ ]:
df_dolbom.merge(df_code, left_on ='행정동', right_on= '읍면동명').drop_duplicates('주소')[['시군구명','행정동','시설명', '주소','Latitude', 'Longitude', '법정동', '행정동코드']]

,시군구명,행정동,시설명,주소,Latitude,Longitude,법정동,행정동코드
0,강남구,일원본동,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,신내동,1168072000
2,강남구,일원본동,강남1호점 우리동네키움센터(강남 다함께키움센터(태화)),서울특별시 강남구 광평로 185,37.484749,127.093420,수서동,1168072000
4,중구,신당제5동,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,37.565088,127.021852,신당동,1114065000
5,중구,약수동,중구4호점 우리동네키움센터(모든아이 약수센터),서울특별시 중구 동호로5길 19,37.551780,127.012577,신당동,1114063500
6,중구,중림동,중구3호점 우리동네키움센터(모든아이 손기정센터),서울특별시 중구 손기정로 101,37.555665,126.964550,만리동1가,1114068000
...,...,...,...,...,...,...,...,...
136,강서구,염창동,강서1호점 우리동네키움센터(염창동),서울특별시 강서구 양천로67길 93-22,37.555989,126.871856,염창동,1150051000
138,강동구,암사제1동,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,암사동,1174057000
139,강남구,세곡동,강남4호점 우리동네키움센터(미미위 세곡 키움센터),서울특별시 강남구 밤고개로 231,37.473781,127.106570,자곡동,1168070000
142,강남구,청담동,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,37.520398,127.054127,청담동,1168056500


- 돌봄시설 수용 인원 크롤링 해야 함

In [ ]:
df_dolbom.to_pickle('/content/drive/My Drive/Project/Seoul/돌봄시설 api/돌봄시설_위치.pickle')